In [1]:
%matplotlib inline
''' Python Tools '''
import numpy as np
np.set_printoptions(suppress=True) # Changes print functionality
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

'''PyTorch Functionality'''
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

'''Landscape Class'''
from Protein_Landscape.landscape_class import Protein_Landscape

'''Dataset Management'''
Datasets = 'Data/'


In [2]:
test = Protein_Landscape(csv_path=Datasets+"Small_NK.csv",gen_graph=True)


Building Protein Graph for entire dataset


100%|██████████| 1000/1000 [00:00<00:00, 84079.46it/s]

Calculating the number of extrema

        Protein Landscape class
            Number of Sequences : 1000
            Max Distance        : 3
            Number of Distances : 3
            Seed Sequence       : AAA
                Modified positions are shown in green
            Number of minima : 1
            Number of maxima : 1
            Normalized Extrema Ruggedness : 0.002
            R/S Ruggedness : 17.319433978287016
        


In [3]:
import cProfile
big = Protein_Landscape(csv_path=Datasets+"NK/K0/V0.csv")
cProfile.run("big.build_graph(single_thread=True)",sort="cumtime")

  1%|          | 598/100000 [00:00<00:33, 2986.47it/s]


        Protein Landscape class
            Number of Sequences : 100000
            Max Distance        : 5
            Number of Distances : 5
            Seed Sequence       : AAAAA
                Modified positions are shown in green
            Number of minima : Not Done
            Number of maxima : Not Done
            Normalized Extrema Ruggedness : Not Done
            R/S Ruggedness : Not Done
        


 23%|██▎       | 23011/100000 [00:07<00:26, 2949.90it/s]

         599452 function calls (599451 primitive calls) in 7.628 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    7.628    7.628 {built-in method builtins.exec}
        1    0.000    0.000    7.628    7.628 <string>:1(<module>)
        1    0.000    0.000    7.628    7.628 landscape_class.py:785(build_graph)
        1    0.077    0.077    7.618    7.618 landscape_class.py:812(<dictcomp>)
    23179    0.048    0.000    7.487    0.000 landscape_class.py:744(calc_neighbours)
    23179    6.297    0.000    6.297    0.000 landscape_class.py:777(<listcomp>)
    23179    0.748    0.000    0.936    0.000 landscape_class.py:717(generate_mutations)
    46357    0.019    0.000    0.324    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
    23178    0.011    0.000    0.207    0.000 <__array_function__ internals>:2(sort)
    23178    0.019    0.000    0.185    0.000 fromnumer

KeyboardInterrupt: 

In [4]:
#cProfile.run("big.build_graph()",sort=cumtime)

In [3]:
big = Protein_Landscape(csv_path=Datasets+"NK/K0/V0.csv",gen_graph=True)

Building Protein Graph for entire dataset


  8%|▊         | 7876/100000 [00:00<00:01, 78756.51it/s]

Calculating the number of extrema


100%|██████████| 100000/100000 [00:01<00:00, 83136.97it/s]


        Protein Landscape class
            Number of Sequences : 100000
            Max Distance        : 5
            Number of Distances : 5
            Seed Sequence       : AAAAA
                Modified positions are shown in green
            Number of minima : 1
            Number of maxima : 1
            Normalized Extrema Ruggedness : 2e-05
            R/S Ruggedness : 30.20584027341869
        


In [4]:
test.calc_neighbours("ACA")

('ACA',
 array([  0,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  30,  40,
         50,  60,  70,  80,  90, 110, 210, 310, 410, 510, 610, 710, 810,
        910]))

In [6]:
test[11]

array(['ACC', 0.419575537582535], dtype=object)